In [1]:
from utilities import *
from env_opt import *
from env_bruteforce import *
from gis.gis_handler import GISHandler

brute_force = True

all_vars = ['x_wec','x_type_wec','x_pen','p_pen','x_env','p_env','p_wec','p_fish_salmon','pos_env', 'gis_handler', 'p_vessel']

conditions = {'current [m/s]': '../data/Surface Current (m-s).tif',
              'oxygen [mg/l]': '../data/Surface Oxygen (mg-l).tif',
              'salinity [PSU]': '../data/Surface Salinity (PSU).tif',
              'temperature [°C]': '../data/Surface Temperature (C).tif',
              'period [s]': '../data/Wave Energy Period (s).tif',
              'height [m]': '../data/Wave Significant Height (m).tif',
              'bathymetry [m]': '../data/Bathymetry (m).tif',
              'distance to port [m]': '../data/Distance to Port (m).tif',
              'distance to shore [m]': '../data/Distance to Shore (m).tif'}

# high fishing is above average, very high is more than one standard deviation above average
conflicts = {'very high fishing traffic': '../data/Very High Fishing Vessel Traffic.geojson',
#            'high fishing traffic': 'data/High Fishing Vessel Traffic.geojson',
             'marine protected areas': '../data/Marine Protected Areas.geojson',
             'danger zones': '../data/Danger Zones and Restricted Areas.geojson',
             'submarine': '../data/Submarine Transit Lanes.geojson',
             'torpex': '../data/Cape Cod TORPEX.geojson',
             'block island': '../data/Block Island Renewable Energy Zone.geojson',
             'ma wind': '../data/MA Wind Energy Areas.geojson',
             'wind lease': '../data/Potential Wind Lease Areas.geojson',
             'wind planning': '../data/Wind Planning Areas.geojson',
             'shipping': '../data/Shipping Lanes.geojson'}

waters = "../data/Northeast State and Federal Waters.geojson"

handler = GISHandler(conditions, conflicts, waters)

args = {}
args['handler'] = handler

if brute_force:
    args['grid_resolution'] = .1  #0.25
    best_x_total, aqua_obj_best_total, p_best, best_x_without_conflict, aqua_obj_best_without_conflict = env_bruteforce(all_vars, args)
    bruteforce_result("without conflict consideration", best_x_total, aqua_obj_best_total, p_best)
    bruteforce_result("with conflict consideration", best_x_without_conflict, aqua_obj_best_without_conflict, p_best)
else:
    x_init, p_init, x, res_best, p_best, op_obj = env_opt(all_vars, args)
    init_result(x_init.nom0, x_init.name, p_init.nom_dict)
    optimize_result(x.name, x.list, x.unit, res_best, p_best)

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
brute force without conflict consideration
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
design variable:
   pos_lat       41.100 m
   pos_long     -69.400 m
----------------------------------------
 objective function terms:
   Objective_func 3888253.633
   fish_yield     1416135.808 [kg]
----------------------------------------
 wave energy converter:
   wec number          6.000 [-]
   wave power          7.189 [kW/m]
   wec_P_ave          15.401 [kW]
   wec AEP        810029.682 [kWh]
----------------------------------------
 price break down:
   wec price            3748500.000 [$]
   pen price            8330210.638 [$]
   fish feed price      3698613.523 [$]
   vessel travel price  139753.633 [$]
----------------------------------------
 location validation:
   conditions           True
   scope                True
   conflicts            True
+++++++++++++++++++++